# T1036.005 - Match Legitimate Name or Location
Adversaries may match or approximate the name or location of legitimate files or resources when naming/placing them. This is done for the sake of evading defenses and observation. This may be done by placing an executable in a commonly trusted directory (ex: under System32) or giving it the name of a legitimate, trusted program (ex: svchost.exe). In containerized environments, this may also be done by creating a resource in a namespace that matches the naming convention of a container pod or cluster. Alternatively, a file or container image name given may be a close approximation to legitimate programs/images or something innocuous.

Adversaries may also use the same icon of the file they are trying to mimic.

## Atomic Tests

### Atomic Test #1 - Execute a process from a directory masquerading as the current parent directory.Create and execute a process from a directory masquerading as the current parent directory (`...` instead of normal `..`)
**Supported Platforms:** macos, linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
mkdir $HOME/...
cp $(which sh) $HOME/...
$HOME/.../sh -c "echo Hello from the Atomic Red Team test T1036.005#1"
```

In [ ]:
Invoke-AtomicTest T1036.005 -TestNumbers 1

#### Cleanup: 
```sh
rm -f $HOME/.../sh
rmdir $HOME/.../
```

In [ ]:
Invoke-AtomicTest T1036.005 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Masquerade as a built-in system executableLaunch an executable that attempts to masquerade as a legitimate executable.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Add-Type -TypeDefinition @'
public class Test {
    public static void Main(string[] args) {
        System.Console.WriteLine("tweet, tweet");
    }
}
'@ -OutputAssembly "$Env:windir\Temp\svchost.exe"

Start-Process -FilePath "$Env:windir\Temp\svchost.exe"
```

In [ ]:
Invoke-AtomicTest T1036.005 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item -Path "$Env:windir\Temp\svchost.exe" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1036.005 -TestNumbers 2 -Cleanup

## Detection
Collect file hashes; file names that do not match their expected hash are suspect. Perform file monitoring; files with known names but in unusual locations are suspect. Likewise, files that are modified outside of an update or patch are suspect.

If file names are mismatched between the file name on disk and that of the binary's PE metadata, this is a likely indicator that a binary was renamed after it was compiled. Collecting and comparing disk and resource filenames for binaries by looking to see if the InternalName, OriginalFilename, and/or ProductName match what is expected could provide useful leads, but may not always be indicative of malicious activity. (Citation: Elastic Masquerade Ball) Do not focus on the possible names a file could have, but instead on the command-line arguments that are known to be used and are distinct because it will have a better rate of detection.(Citation: Twitter ItsReallyNick Masquerading Update)

In containerized environments, use image IDs and layer hashes to compare images instead of relying only on their names.(Citation: Docker Images) Monitor for the unexpected creation of new resources within your cluster in Kubernetes, especially those created by atypical users.